# WorkShop 2

Let's play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [16]:

import pandas as pd
import numpy as np
import re

import numpy as np

def convert_float(n: float) -> np.float64:
    """
    Convert the input to a numpy float64.

    Parameters:
    n (float): The input value to convert.

    Returns:
    np.float64: The converted float value, or np.NAN if conversion fails.
    """
    try:
        return np.float64(n)
    except Exception as e:
        return np.NAN

def convert_int(n: int) -> np.float64:
    """
    Convert the input to a numpy int32 multiplied by 1,000,000.

    Parameters:
    n (int): The input value to convert.

    Returns:
    np.float64: The converted int value, or np.NAN if conversion fails.
    """
    try:
        return np.int32(n) * 1000000
    except Exception as e:
        return np.NAN


def generate_energy_df() ->pd.DataFrame:
    """
    Generate a DataFrame containing energy indicators data.

    Reads data from an Excel file, cleans country names, and applies conversions to certain columns.

    Returns:
    pd.DataFrame: A DataFrame with energy indicators data.
    """
    energy_indicators_cols=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy_df =pd.read_excel(
                     './assets/Energy Indicators.xls',  # Asegúrate de que la ruta sea correcta
                     engine='xlrd',  # Usa 'openpyxl' para archivos .xlsx
                     sheet_name='Energy',  # Usar el nombre correcto de la hoja
                     names=energy_indicators_cols,
                     header=None,
                     skiprows=list(range(18)),
                     skipfooter=38,
                     usecols=[2,3,4,5],
                     converters={'energy_df Supply':convert_int, 
                                 'energy_df Supply per Capita':convert_float,
                                 '% Renewable':convert_float
                                }
                    )
    
    energy_df['Country'] = energy_df['Country'].apply(lambda x: re.sub(r"\s?\(.*?\)", "", x))
    energy_df['Country'] = energy_df['Country'].apply(lambda x: re.sub(r"^\d+|\d+$", "", x))
   
    energy_df= energy_df.replace({"Country":{r"^(Republic of Korea).*": "South Korea",
                               r'^(United States of America).*':'United States',
                               r"^(United Kingdom of Great Britain and Northern Ireland).*": "United Kingdom",
                               r"^(China, Hong Kong Special Administrative Region).*": "Hong Kong"
                              }},regex=True)
    return energy_df
def get_GDP_df() -> pd.DataFrame:
    """
    Generate a DataFrame containing GDP data.

    Reads data from a CSV file, renames columns, and standardizes country names.

    Returns:
    pd.DataFrame: A DataFrame with GDP data.
    """
    gdp_df = pd.read_csv('./assets/world_bank.csv',
                    header = None, skiprows = list(range(4)))
    gdp_df.columns = gdp_df.iloc[0].astype(str)
    gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)
    gdp_df['Country'].replace({"Korea, Rep.": "South Korea", 
                            "Iran, Islamic Rep.": "Iran", 
                            "Hong Kong SAR, China": "Hong Kong"}, inplace = True)
    gdp_df.rename(columns=lambda x: re.sub(r'\.0$', '', str(x)), inplace=True)
    return gdp_df

def get_scimago_df()-> pd.DataFrame:
    """
    Generate a DataFrame containing SCImago Journal Rank (SJR) data.

    Reads data from an Excel file.

    Returns:
    pd.DataFrame: A DataFrame with SCImago Journal Rank data.
    """
    scimago_df = pd.read_excel('./assets/scimagojr-3.xlsx')
    return scimago_df

def answer_one()-> pd.DataFrame:
    """
    Generate a DataFrame containing data merged from energy, GDP, and SCImago datasets.

    The function merges data from three sources, filters the top 15 ranked countries by SCImago,
    and selects specific columns.

    Returns:
    pd.DataFrame: A DataFrame with merged data for the top 15 ranked countries.
    """
    energy_df=generate_energy_df()
    gdp_df=get_GDP_df()
    scimago_df=get_scimago_df()
    join_df = pd.merge(pd.merge(energy_df, gdp_df, on='Country', how='inner'), scimago_df, on='Country', how='inner')
    join_df.set_index('Country', inplace=True)
    join_df = join_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    join_df=join_df[join_df['Rank'].isin(range(1, 16))]
    join_df = join_df.sort_values(by='Rank', ascending=True)
    
    return join_df

In [17]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"
assert list(answer_one().columns)==['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'], 'not the correct columns'
print(answer_one().head(15))

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       889

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


In [ ]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [1]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [18]:
def answer_two()-> int:
    """
    Calculate the number of countries in the union of the three datasets but not in their intersection.

    The function merges the energy, GDP, and SCImago datasets with an outer join, 
    and compares it to the intersection of these datasets from answer_one.

    Returns:
    int: The number of countries in the union of the datasets but not in their intersection.
    """
    full_union = pd.merge(pd.merge(generate_energy_df(), get_GDP_df(), on='Country', how='outer'), get_scimago_df(), on='Country', how='outer')
    intersect = answer_one()
    return len(full_union)-len(intersect)

In [20]:
# example of test cases
print(answer_two())
assert type(answer_two()) == int, "Q2: You should return an int number!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

305


In [ ]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [21]:
def answer_three()-> pd.Series:
    """
    Calculate the average GDP over the years 2006-2015 for the top 15 countries by rank,
    and sort them in descending order.

    The function calculates the mean GDP for each country over the specified years,
    sorts the values in descending order, and returns the result.

    Returns:
    pd.Series: A Series with the average GDP of the top 15 countries, sorted in descending order.
    """
    top15_gdp = answer_one()
    top15_gdp['GDP_avg'] = top15_gdp[['2006','2007','2008','2009','2010','2011',
                            '2012','2013','2014','2015']].mean(axis = 1)
    avg_gdp = top15_gdp['GDP_avg'].sort_values(ascending = False)
    return avg_gdp

In [50]:
# example of test cases
print(answer_three())
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: GDP_avg, dtype: float64


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [51]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [52]:
def answer_four()-> float:
    """
    Calculate the difference in GDP between 2006 and 2015 for the 6th country 
    in the list of top 15 countries sorted by average GDP over the years 2006-2015.

    The function calculates the mean GDP for each country over the specified years,
    sorts the countries by average GDP in descending order, and returns the GDP difference 
    between 2015 and 2006 for the 6th country in this sorted list.

    Returns:
    float: The difference in GDP between 2015 and 2006 for the 6th country in the sorted list.
    """

    top15_gdp = answer_one()
    top15_gdp['GDP_avg'] = top15_gdp[['2006','2007','2008','2009','2010','2011',
                            '2012','2013','2014','2015']].mean(axis = 1)
    top15_gdp.sort_values(by='GDP_avg',ascending=False)
    final = top15_gdp.iloc[5]['2015']
    initial = top15_gdp.iloc[5]['2006']
    return float(final - initial)


In [53]:
# example of test cases
print(answer_four())
assert type(answer_four()) == float, "Q4: You should return a single number!"


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


228139911279.3601


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


In [54]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [55]:
def answer_five()-> float:
    """
    Calculate the mean energy supply per capita across the top 15 countries by rank.

    The function calculates the mean energy supply per capita for the top 15 countries 
    by rank and returns the result as a float.

    Returns:
    float: The mean energy supply per capita across the top 15 countries.
    """
    top15_gdp = answer_one()
    return float(top15_gdp['Energy Supply per Capita'].mean())

In [56]:
# example of test cases
print(answer_five())
assert type(answer_five()) == float, "Q5: You should return a single number!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

157.6


In [57]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [58]:
def answer_six()-> tuple:
    """
    Identify the country with the highest percentage of renewable energy (% Renewable).

    The function identifies the country with the highest percentage of renewable energy 
    (% Renewable) among the top 15 countries by rank, and returns a tuple containing 
    the country name and the corresponding % Renewable value.

    Returns:
    tuple: A tuple containing the country name and the highest % Renewable value.
    """ 
    top15_gdp = answer_one()
    re_max_country = (top15_gdp['% Renewable'].idxmax(), 
                      top15_gdp['% Renewable'].max())
    return re_max_country

In [59]:
# example of test cases
print(answer_six())
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


('Brazil', 69.64803)


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [60]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [61]:
def answer_seven()-> tuple:
    """
    Identify the country with the highest ratio of self-citations to total citations.

    The function calculates the ratio of self-citations to total citations for each country 
    in the top 15 countries by rank, identifies the country with the highest ratio, 
    and returns a tuple containing the country name and the corresponding ratio value.

    Returns:
    tuple: A tuple containing the country name and the highest ratio of self-citations to total citations.
    """
    top15_gdp=answer_one()
    top15_gdp['radio_self_citations']=top15_gdp['Self-citations']/top15_gdp['Citations']
    return (top15_gdp['radio_self_citations'].idxmax(),top15_gdp['radio_self_citations'].max())

In [62]:
# example of test cases
print(answer_seven())
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

('China', 0.6893126179389422)


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


In [63]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [64]:
def answer_eight()-> str:
    """
    Identify the third most populous country based on energy supply and energy supply per capita.

    The function calculates the estimated population for each country in the top 15 countries 
    by rank based on energy supply and energy supply per capita, identifies the country 
    with the third highest estimated population, and returns its name.

    Returns:
    str: The name of the third most populous country based on energy supply and energy supply per capita.
    """

    top15_gdp = answer_one()
    top15_gdp['population_estimate'] = top15_gdp['Energy Supply']/top15_gdp['Energy Supply per Capita']
    population = top15_gdp['population_estimate'].sort_values(ascending = False).index[2]
    return population

In [65]:
# example of test cases
print(answer_eight())
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


United States


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


In [66]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [67]:
def answer_nine()-> float:
    """
    Calculate the correlation between citable documents per capita and energy supply per capita.

    The function calculates the correlation coefficient between the number of citable documents 
    per capita and energy supply per capita for each country in the top 15 countries by rank, 
    and returns the correlation coefficient as a float.

    Returns:
    float: The correlation coefficient between citable documents per capita and energy supply per capita.
    """
    top15_gdp = answer_one()
    top15_gdp['population_estimate'] = top15_gdp['Energy Supply']/top15_gdp['Energy Supply per Capita']
    top15_gdp['Citable docs per Capita'] = top15_gdp['Citable documents']/top15_gdp['population_estimate']
    correlation = top15_gdp['Citable docs per Capita'].corr(top15_gdp['Energy Supply per Capita'])
    return correlation

In [68]:
# example of test cases
print(answer_nine())
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


0.7940010435442945


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [69]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [70]:

def answer_ten()-> pd.Series:
    """
    Identify countries with high renewable energy (% Renewable) compared to the median.

    The function calculates the median of % Renewable energy for the top 15 countries 
    by rank, assigns a binary indicator (1 for high, 0 for low) based on whether 
    % Renewable is above or below the median, and returns a Series containing 
    these binary indicators sorted by rank.

    Returns:
    pd.Series: A Series containing binary indicators (1 or 0) for high renewable energy, 
               sorted by rank of the countries.
    """
    top15_gdp = answer_one()
    renewable_media = top15_gdp["% Renewable"].median(axis=0)
    top15_gdp["high_renew"] = top15_gdp.apply(lambda x: 1 if x["% Renewable"] > renewable_media else 0,axis=1)
    top15_gdp.sort_values(by='Rank', inplace=True)
    return top15_gdp["high_renew"] 

In [71]:
# example of test cases
print(answer_ten())
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"
assert answer_ten().isin([0, 1]).all(),'the values of the columns are not the indicated'

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: high_renew, dtype: int64


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [72]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [73]:

def answer_eleven()-> pd.DataFrame:
    """
    Calculate population statistics grouped by continent for the top 15 countries by GDP rank.

    The function assigns each country to a continent based on a predefined dictionary,
    calculates the estimated population for each country, groups the data by continent,
    and computes the size (number of countries), total population, mean population,
    and standard deviation of population for each continent.

    Returns:
    pd.DataFrame: A DataFrame with population statistics grouped by continent.
    """
    top15_gdp = answer_one()
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}

    top15_gdp['Continent'] = top15_gdp.index.map(ContinentDict)
    top15_gdp['population_estimate'] = top15_gdp['Energy Supply'] / top15_gdp['Energy Supply per Capita']
    
    result = top15_gdp.groupby('Continent')['population_estimate'].agg(['size', 'sum', 'mean', 'std'])
    return result
   

In [74]:
# example of test cases
print(answer_eleven())
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

               size          sum        mean         std
Continent                                               
Asia              5  2898.666387  579.733277  679.097888
Australia         1    23.316017   23.316017         NaN
Europe            6   457.929667   76.321611   34.647667
North America     2   352.855249  176.427625  199.669645
South America     1   205.915254  205.915254         NaN


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",


In [75]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [76]:
def answer_twelve()-> pd.Series:
    """
    Calculate the number of countries in each % Renewable bin grouped by continent.

    The function assigns each country to a continent based on a predefined dictionary,
    categorizes the countries into 5 bins based on % Renewable, and counts the number 
    of countries in each bin for each continent.

    Returns:
    pd.Series: A Series with the count of countries in each % Renewable bin 
               grouped by continent.
    """
    top15_gdp = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    top15_gdp["bins"] = pd.cut(top15_gdp["% Renewable"], 5)
    result=top15_gdp.groupby([ContinentDict, top15_gdp['bins']]).size()
    
    return result

In [77]:
# example of test cases
print(answer_twelve())
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"

C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\2231204751.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result=top15_gdp.groupby([ContinentDict, top15_gdp['bin

Country        bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\2231204751.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result=top15_gdp.groupby([ContinentDict, top15_gdp['bin

In [78]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [96]:
def answer_thirteen()->pd.Series:
    """
    Format the estimated population for each country using dots.

    The function calculates the estimated population for each country based on
    energy supply and energy supply per capita, and formats the population estimate
    with thousands separators.

    Returns:
    pd.Series: A Series with the formatted population estimate for each country.
    """
    top15_gdp = answer_one()
    top15_gdp['population_estimate'] = top15_gdp['Energy Supply'] / top15_gdp['Energy Supply per Capita']
    top15_gdp['population_estimate'] = top15_gdp['population_estimate'].apply(lambda x: '{:,}'.format(x).replace(',', 'x').replace('.', ',').replace('x', '.'))
    return top15_gdp['population_estimate']

In [97]:
# example of test cases
print(answer_thirteen())
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

Country
China                 1.367,6451612903227
United States          317,61538461538464
Japan                  127,40939597315436
United Kingdom          63,87096774193548
Russian Federation                  143,5
Canada                  35,23986486486486
Germany                 80,36969696969697
India                 1.276,7307692307693
France                  63,83734939759036
South Korea             49,80542986425339
Italy                  59,908256880733944
Spain                  46,443396226415096
Iran                    77,07563025210084
Australia              23,316017316017316
Brazil                 205,91525423728814
Name: population_estimate, dtype: object


C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdp_df['Country'].replace({"Korea, Rep.": "South Korea",
C:\Users\57300\AppData\Local\Temp\ipykernel_17400\3791049128.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [81]:
# space for proffesor tests